<a href="https://colab.research.google.com/github/sclaudiobr/algoritmos-py/blob/master/filtragem02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Filtragem com máscaras

As técnicas de filtragem são transformações da imagem pixel a pixel, que não dependem apenas do nível de cinza de um determinado pixel, mas também do valor dos níveis de cinza dos pixels vizinhos. O processo de filtragem é feito utilizando matrizes denominadas máscaras, as quais são aplicadas sobre a imagem cujo a resultante é uma nova imagem transformada. A finalidade de aplicação de uma máscara é ralçar feições exitentes nas imagens com o intuito de obter melhor detales sobre os elementos contidos facilitando a classificação e segmentação dos objetos. Neste processo as imagens são convertidas em Arrays (arranjos de matrizes) e processadas atraves do numpy, biblioteca dedica as operações matriciais com diversas funções. Posteriormente as matrizes são revertidas para imagens georreferenciadas usando as referências capturadas da próparia imagem original. Para executar este tutorial siga atentamente as dicas sobre cada bloco de código do algoritmo, com especial atenção ao bloco que terá de informar o filtro a ser aplicado. Ao final será exportado uma imagem convertida denominada "FILTRADA.TIF" na aba Files no canto superior esquerdo.

@ autor Claudio Roberto Oliveira da Silva 
<p>MSc. Modelagem Computacional, Eng. Civil - COPPE/UFRJ<p>
Curriculo Lattes: <a href="http://lattes.cnpq.br/7294686843905076">Lattes</a>
<p>
Repositório: <a href="https://github.com/sclaudiobr/"> GitHub</a>

In [2]:
!pip install Pillow # <-- execute esta linha apenas a primeira vez

In [0]:
# carregando bibliotecas < -- apenas uma vez
from osgeo import ogr, gdal, osr
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import PIL as pil
import scipy.signal
from skimage import io, color
from skimage import exposure
import urllib.request

In [4]:
# Baixando o arquivo < -- apenas uma vez
url = ('https://github.com/sclaudiobr/algoritmos-py/blob/master/planetario.tar.xz?raw=true')
filename, headers = urllib.request.urlretrieve(url, filename="planetario.tar.xz")
print ("download completo!")
print ("Nome do arquivo: ", filename)

download completo!
Nome do arquivo:  planetario.tar.xz


In [5]:
print ("descompactando...")#  < --- apenas uma vez
!tar -Jxxvf planetario.tar.xz

descompactando...
planetario.tif


In [0]:
# ---- Apenas uma vez <------
img = io.imread('planetario.tif')   # Abrindo a imagem
img = color.rgb2gray(img)       # Converte a imagem para escala de cinza (1 canal)
GDAL = gdal.Open('planetario.tif') # Abrindo o Geotiff


# Arquivo de destino
path_GDALB = 'FILTRADA.TIF'

# PEGANDO DADOS DE REFERENCIAS GEOGRÁFICAS DA IMAGEM GEOTIF 

# Processo necessário para reprojetar de volta a matriz obtida com os calculos
projection = GDAL.GetProjection() # <- pega aprojeção
geotransform = GDAL.GetGeoTransform()# <- Associa as coordenadas
originX,pixelWidth,empty,finalY,empty2,pixelHeight=geotransform # <-- faz a conversão de projeção
cols =  GDAL.RasterXSize # <-- guarda valores do eixo X
rows =  GDAL.RasterYSize # <-- guarda valores do eixo Y
finalX = originX + pixelWidth * cols
originY = finalY + pixelHeight * rows
extentArray = [originX,finalX,originY,finalY]


# CRIANDO AS MARCARAS 3X3

#Filtros passa baixa
passa_baixa1 = np.array([[1,1,1],[1,2,1],[1,1,1]])/9 # 1/10
passa_baixa2 = np.array([[1,2,1],[2,4,2],[1,2,1]])/16 # 1/16
# Filtros ineares passa alta
passa_alta1 = np.array([[0,-1,0],[-1,5,-1],[0,-1,0]])
passa_alta2 = np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])
passa_alta3 = np.array([[1,-2,1],[1,5,1],[1,-2,1]])
# Filtro de bordas direcional
dirN  = np.array([[1,1,1],[1,-2,1],[-1,-1,-1]]) # filtro direcional Norte
dirNE = np.array([[1,1,1],[-1,-2,1],[-1,-1,1]]) # filtro direcional Nordeste
dirL  = np.array([[-1,1,1],[-1,-2,1],[-1,1,1]]) # filtro direcional Leste
dirSE = np.array([[-1,-1,1],[-1,-2,1],[1,1,1]]) # filtro direcional Sudeste
dirS  = np.array([[-1,-1,-1],[1,-2,1],[1,1,1]]) # filtro direcional Sul
dirSW = np.array([[1,-1,-1],[1,-2,-1],[1,1,1]]) # filtro direcional Sudoeste
dirW  = np.array([[1,1,-1],[1,-2,-1],[1,1,-1]]) # filtro direcional Oeste
dirNW = np.array([[1,1,1],[1,-2,-1],[1,-1,-1]]) # filtro direcional Noroeste
# Filtro não linear de bordas
bordaA = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]])
bordaM = np.array([[ 0,-1, 0],[-1 , 4,-1],[0,-1,0]])
bordaB = np.array([[ 1,-2, 1],[-2, 3,-2], [1,-2,1]])
# Filtro não linear
sobela = np.array([[-1,2,-1],[ 0,0,0],[ 1,2,1]])
sobelb = np.array([[-1, 0,1],[-2,0,2],[-1,0,1]])
sobel = np.array(np.sqrt(sobela**2+sobelb**2))
# Filtros morfológicos
mediano = np.array([[1,1,1],[1,1,1],[1,1,1]])
xis     = np.array([[1,0,1],[0,1,0],[1,0,1]])
cruz    = np.array([[0,1,0],[1,1,1],[0,1,0]])
hor     = np.array([[0,0,0],[1,1,1],[0,0,0]])
vert    = np.array([[0,1,0],[0,1,0],[0,1,0]])
diag1   = np.array([[0,0,1],[0,1,0],[1,0,0]])
diag2   = np.array([[1,0,0],[0,1,0],[0,0,1]])
# Fonte: <http://www.dpi.inpe.br/spring/portugues/tutorial/filtragem.html>

In [0]:
# O FILTRO É ALTERADO NA LINHA ABAIXO --->      Ex: filtro = <AQUI>
filtro = hor         # <-- AQUI

In [0]:
# ---- FUNÇÔES ---
# ---- Filtro -- equalização -- Salvamento e Projeção
# Aplicação do filtro com a máscara escolhida acima
def frealce(filtro):
  filtro = scipy.signal.convolve2d(img, filtro, 'valid') 
  return filtro
filtrada = frealce(filtro)

# Ajuste do contraste da imagem filtrada com aplicação de uma Equalização de histograma
def equImage(filtrada):
  equalizada = exposure.equalize_adapthist(filtrada/np.max(np.abs(filtrada)), clip_limit=0.03)
  return equalizada

FILTRADA = equImage(filtrada)

# ----- Salvando os dados
def saveRaster(dataset,datasetPath,cols,rows,projection):
    rasterSet = gdal.GetDriverByName('GTiff').Create(datasetPath, cols, rows,1,gdal.GDT_Float32)
    rasterSet.SetProjection(projection)
    rasterSet.SetGeoTransform(geotransform)
    rasterSet.GetRasterBand(1).WriteArray(dataset)
    rasterSet.GetRasterBand(1).SetNoDataValue(-999)
    rasterSet = None
saveRaster(FILTRADA,path_GDALB,cols,rows,projection)

# ----- Plotando so dados
def plotRaster(rasterImage,extentArray,vmin,cmap):
    raster = gdal.Open(rasterImage)
    dsraster = raster.ReadAsArray()
    plt.figure(figsize=(20,20))
    im = plt.imshow(dsraster, vmin=vmin, cmap=cmap, extent=extentArray)#
    plt.colorbar(im, fraction=0.015)
    plt.xlabel('Leste', size=20,color='white')
    plt.ylabel('Sul', size=20,color='white',  rotation=90)
    plt.show()
plotRaster(path_GDALB,extentArray,0,'gray')
